In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from ruccent import Accent
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import openpyxl

In [ ]:
%matplotlib inline
#Функции для обработки слова из "слова" --> ['с', 'л', 'О', 'в', 'а']
def word_maker(word: str) -> list[str]:
    """Главная функция разделения слова и его подготовки"""
    word = mark_softness(re.sub(r'[^а-яё]', '', word.lower()))
    word = mark_stress(word)
    return word

def mark_stress(lst_word: list[str]) -> str:
    """Указывает ударение в русском слове."""
    accenter = Accent()
    word = ''.join(lst_word).replace("'", '')
    index = accenter.predict(word)
    lst_word[index] = chr(ord(lst_word[index])-32)
    return lst_word

def mark_softness(word: str) -> list[str]:
    """Указывает в слове мягкие согласные"""
    letters = []

    for i in range(1, len(word)):
        if (word[i] in 'яёеюиь' and word[i-1] not in 'ьъщжшцеуэоаыяию') or word[i-1] in 'йчщ':
            letters.append(word[i-1]+"'")
        else:
            letters.append(word[i-1])
    if word[-1] in 'йчщ':
        letters.append(word[-1] + "'")
    else:
        letters.append(word[-1])
    return [i for i in letters if i not in 'ьъ']


In [ ]:
#тест на каких-то рандомных словах
lst = ['воробей', 'Котенок', 'розовый', 'Карандаш', 'девочка', 'Морда', 'Море', 'конфетка']
for word in lst:
    print(f'{word} - {word_maker(word)}'

In [ ]:
#Функции подготовки датасета, разбор слова, считывание excel, csv и запись#всех данных в одну csv таблицу.
def examples(word):
  """Мы учитываем коэффициенты букв, их частоты, ударения и позицию, также мы смотрим на мягкость каждой буквы."""

  pd_values, pd_frequency = get_data()

  word = word_maker(word)

  k_total = 0   # (mx/freq) - ki
  ki = 0
  f_total = 0   # val - fi

  max_frequency = max([pd_frequency['values'][let] for let in word])
  for i in range(len(word)):
      if not i:
          ki = 4 * (max_frequency / pd_frequency['values'][word[i]])
      elif word[i] in 'УЕЫАОЭЮИЯ':
          ki = 2 * (max_frequency / pd_frequency['values'][word[i]])
      else:
          ki = (max_frequency / pd_frequency['values'][word[i]])
      f_total += ki*pd_values['values'][word[i].lower()]
      k_total += ki

  return f_total, k_total

def get_data():
  '''Функция считывает данные для подготовки слова (частоту и значения)'''
  pd_values = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/letters.csv', index_col='letters')
  pd_frequency = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/frequency.csv', index_col='letters')
  return pd_values, pd_frequency

def get_excel():
  '''Функция достаёт из excel таблицы нужные данные и возвращает словарь из них'''
  dct = {}
  book = openpyxl.open('words.xlsx', read_only=True)
  sheet1 = book.active
  for col in range(1, 89):
      dct[sheet1[col][0].value] = float(sheet1[col][1].value)
  return dct

# Нежный-Грубый
def create_dataset():
  '''Функция создаёт нужный датасет с данными  - слово, y, 1ый параметр(f), 2ой параметр(k), и то, что мы могли бы посчитать сами(f/k)'''
  f_lst = []
  k_lst = []
  fk_lst = []
  df = pd.read_csv('DataSet0_1.csv', encoding='UTF-8')
  rows = df.shape[0]

  for i in range(0, rows):
      f, k = examples(df['words'][i])
      f_lst.append(f)
      k_lst.append(k)
      fk_lst.append(round(f/k, 3))

  df['f'] = f_lst
  df['k'] = k_lst
  df['f/k'] = fk_lst

  df.to_csv('NewDataSet0_1.csv')


In [ ]:
def get_text():
  '''Достаёт нужные данные из заготовленной таблицы вида - слово, y'''
  texts = []
  book = openpyxl.open('Data.xlsx', read_only=True)
  sheet = book.active

  for row in range(1, sheet.max_row+1):
      texts.append(sheet[row][0].value)
  return texts

def get_csv():
  '''Делает из excel --> csv'''
  excel = get_excel()
  df = pd.DataFrame({
      'words': excel.keys(),
      'y': excel.values()})
  df.to_csv('DataSet0_1.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NewDataSet0_1.csv', index_col=0, encoding='UTF-8')
df.head(10)

Визуализация

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['f/k'], ec='black', color='#2196f3')
plt.xlabel('ниже 2.5 нежный, выше 3.5 грубый')
plt.ylabel('количество слов')
plt.show()

plt.figure(figsize=(10, 6))
plt.hist(df['k'], ec='black', color='#2196f3')
plt.xlabel('Разбаловка k')
plt.ylabel('количество слов')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['f'], ec='black', color='#2196f3')
plt.xlabel('Разбаловка f')
plt.ylabel('количество слов')
plt.show()

In [ ]:
sns.distplot(df['f'])
plt.title(f'F')
plt.show()


In [ ]:
sns.distplot(df['k'])
plt.title(f'K')
plt.show()


In [ ]:
df.describe()

In [ ]:
# Кореляция

df['y'].corr(df['f/k'])
df['y'].corr(df['f'])
df['y'].corr(df['k'])
df.corr()

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(), annot=True, annot_kws={"size": 14})
sns.set_style('white')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()


In [ ]:
plt.scatter(df['f'], df['k'])
z = np.polyfit(df['f'], df['k'], 1)
p = np.poly1d(z)
plt.plot(df['f'], p(df['f']))
plt.show()


Множественная Линейная Регрессия

In [ ]:
data = df[['f', 'k']].values
target = df[['y']].values
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=15)
LinReg = LinearRegression()
LinReg.fit(X_train, y_train)

print('Training data score:', LinReg.score(X_train, y_train))
print('Test data score:', LinReg.score(X_test, y_test))
print('Intercept:', *LinReg.intercept_)


In [ ]:
pd.DataFrame(data=LinReg.coef_, columns=['f', 'k'], index=['coef'])

Трансформируем параметры

In [ ]:
k_log = np.log(df['k'])
f_log = np.log(df['f'])

sns.distplot(k_log)
plt.title(f'Log f with skew {k_log.skew()}')
plt.show()

sns.distplot(f_log)
plt.title(f'Log f with skew {f_log.skew()}')
plt.show()


Модель, которая использует логарифмы f и k, как основные параметры.

In [ ]:
df['k_log'] = k_log
df['f_log'] = f_log
data = df[['k_log', 'f_log']]
target = df[['y']].values
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=15)
LinReg = LinearRegression()
LinReg.fit(X_train, y_train)

print('Training data score:', LinReg.score(X_train, y_train))
print('Test data score:', LinReg.score(X_test, y_test))
print('Intercept:', *LinReg.intercept_)


In [ ]:
pd.DataFrame(data=LinReg.coef_, columns=['f', 'k'], index=['coef'])

In [ ]:
df.head()

In [ ]:
def predict_check():
    words = ['розовый', 'тюльпан', 'гвоздь', 'день', 'никита', 'юбка', 'бутылка', 'животное']
    x_lst = []
    y_lst = [1, 0, 1, 0, 0, 0, 1, 1]
    for word in words:
        x_lst.append(examples(word))
    return words, x_lst, y_lst


In [ ]:
words, X_predict, y_predict = predict_check()
y_predicted = LinReg.predict(np.log(X_predict))
for i in range(len(words)):
  print(f"Predict for '{words[i]}' - {1 if -3.5 > y_predicted[i] else 0}, should be - {y_predict[i]}")


In [ ]:
plt.scatter(df['f_log'], df['k_log'])
z = np.polyfit(df['f_log'], df['k_log'], 1)
p = np.poly1d(z)
plt.plot(df['f_log'], p(df['f_log']))
plt.show()
